# Re-balancing the combined data - loss / gain. 

7/28/17

- we want to run new subjects with loss/gain combined. 
- the issues: 
    - as of now are that the ambiguous trials. At most levels of ambiguity, there are imbalances either away from ambiguous (gain) or for it (loss)
    - the imbalances are the opposite in the gain and loss trials. 
    
- solution: 
    - for each level of ambiguity, adjust some of the magnitudes. 


In [2]:
import pandas as pd
import numpy as np
import os
np.random.seed(1000)

# Functions

In [4]:

def add_columns(trial_table):


    trial_table['info_l'] = (trial_table.revealed_x_l + trial_table.revealed_o_l)/50.0
    trial_table['info_r'] = (trial_table.revealed_x_r + trial_table.revealed_o_r)/50.0

    # calculating prob of outcome 
    trial_table['prob_out_l'] = trial_table.revealed_x_l / (trial_table.revealed_x_l + trial_table.revealed_o_l)
    trial_table['prob_out_r'] = trial_table.revealed_x_r / (trial_table.revealed_x_r + trial_table.revealed_o_r)
    trial_table['prob_out_l_bayes'] = trial_table['prob_out_l'] # for non-ambiguous trials, use the regular probabilities
    trial_table['prob_out_r_bayes'] = trial_table['prob_out_r']
    # adjusting for bayes 
    trial_table.loc[trial_table['info_l']!=1.0,'prob_out_l_bayes'] = (trial_table.revealed_x_l +1) / (trial_table.revealed_x_l + trial_table.revealed_o_l + 2) # for ambiguous trials, use the posterior expectation on uniform prior.
    trial_table.loc[trial_table['info_r']!=1.0,'prob_out_r_bayes'] = (trial_table.revealed_x_r +1) / (trial_table.revealed_x_r + trial_table.revealed_o_r + 2)

    trial_table['ambig_l']=(trial_table['info_l']!=1.).astype('int')
    trial_table['ambig_r']=(trial_table['info_r']!=1.).astype('int')

    # selectors
    amb_l = trial_table['ambig_l']==1
    amb_r = trial_table['ambig_r']==1
    both_unamb = (trial_table['ambig_l']==0) & (trial_table['ambig_r']==0)
    amb = (trial_table['ambig_l']==1) | (trial_table['ambig_r']==1)


    # start with Nan
    trial_table['prob_out_ambig'] = np.nan
    trial_table['prob_out_unambig'] = np.nan
    trial_table['mag_ambig'] = np.nan
    trial_table['mag_unambig'] = np.nan
    trial_table['resp_amb_1'] = np.nan

    # create probability of o on ambiguous trials
    trial_table.loc[amb_l,'prob_out_ambig']=trial_table.revealed_x_l[amb_l]/(trial_table.revealed_x_l[amb_l] + trial_table.revealed_o_l[amb_l])
    trial_table.loc[amb_r,'prob_out_ambig']=trial_table.revealed_x_r[amb_r]/(trial_table.revealed_x_r[amb_r] + trial_table.revealed_o_r[amb_r])


    # create bayesian adjusted probabilities
    trial_table.loc[amb_l,'prob_out_ambig_bayes']=(trial_table.revealed_x_l[amb_l]+1)/(trial_table.revealed_x_l[amb_l] + trial_table.revealed_o_l[amb_l]+2)
    trial_table.loc[amb_r,'prob_out_ambig_bayes']=(trial_table.revealed_x_r[amb_r]+1)/(trial_table.revealed_x_r[amb_r] + trial_table.revealed_o_r[amb_r]+2)

    # create proabbility on umambiguous (switch index right v left e.g. left revealed when right is ambiguous)
    trial_table.loc[amb_r&amb,'prob_out_unambig']=trial_table.revealed_x_l[amb_r&amb]/(trial_table.revealed_x_l[amb_r&amb] + trial_table.revealed_o_l[amb_r&amb])
    trial_table.loc[amb_l&amb,'prob_out_unambig']=trial_table.revealed_x_r[amb_l&amb]/(trial_table.revealed_x_r[amb_l&amb] + trial_table.revealed_o_r[amb_l&amb])
    trial_table.loc[:,'info_amb']  = np.minimum(trial_table['info_l'].as_matrix(),trial_table['info_r'].as_matrix())

    # create magnitutde on ambiguous trials
    trial_table.loc[amb_l,'mag_ambig']=trial_table.mag_left[amb_l]
    trial_table.loc[amb_r,'mag_ambig']=trial_table.mag_right[amb_r]
    trial_table.loc[amb_r&amb,'mag_unambig']=trial_table.mag_left[amb_r&amb]
    trial_table.loc[amb_l&amb,'mag_unambig']=trial_table.mag_right[amb_l&amb]
    return trial_table


In [5]:
# same but for printed diffferently 
def print_balance(trial_table):
    print('(mag, prob outcome)\n')
    print('gain')
    tt=trial_table.loc[(trial_table['mag_right']>0),].copy()
    for revealed in np.sort(tt['info_amb'].unique()):
        print('ambig prop urns revealed:{0}').format(str(revealed))
        mag_ambig = np.round(tt.loc[tt['info_amb']==revealed,'mag_ambig'].mean(),2)
        mag_unambig = np.round(tt.loc[tt['info_amb']==revealed,'mag_unambig'].mean(),2)
        prob_ambig = np.round(tt.loc[tt['info_amb']==revealed,'prob_out_ambig_bayes'].mean(),2)
        prob_unambig = np.round(tt.loc[tt['info_amb']==revealed,'prob_out_unambig'].mean(),2)
        print('ambig v.s. unambig: ({0},{1}) v.s. ({2},{3})').format(mag_ambig,prob_ambig,mag_unambig,prob_unambig)
        print('')
    
    print('loss')
    tt=trial_table.loc[(trial_table['mag_right']<0),].copy()
    for revealed in np.sort(tt['info_amb'].unique()):
        print('ambig prop urns revealed:{0}').format(str(revealed))
        mag_ambig = np.round(tt.loc[tt['info_amb']==revealed,'mag_ambig'].mean(),2)
        mag_unambig = np.round(tt.loc[tt['info_amb']==revealed,'mag_unambig'].mean(),2)
        prob_ambig = np.round(tt.loc[tt['info_amb']==revealed,'prob_out_ambig_bayes'].mean(),2)
        prob_unambig = np.round(tt.loc[tt['info_amb']==revealed,'prob_out_unambig'].mean(),2)
        print('ambig v.s. unambig: ({0},{1}) v.s. ({2},{3})').format(mag_ambig,prob_ambig,mag_unambig,prob_unambig)
        print('')

# Checking the original balance



In [157]:
trial_table = pd.read_csv('parameters/Parameters_block_gain_loss_combined_recorded_from_subject.csv')
trial_table=add_columns(trial_table)

In [158]:
trial_table.head(10)

,id,MID,trial_number,start_date,trialstart,outcome,reaction_time,totalpoints,prop_left,prop_right,...,prob_out_r_bayes,ambig_l,ambig_r,prob_out_ambig,prob_out_unambig,mag_ambig,mag_unambig,resp_amb_1,prob_out_ambig_bayes,info_amb
0,77666,A27BPRHBM28XZS,0,3/10/17 8:43,1.489140e+12,X,2642,5000,0.16,0.76,...,0.260000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.00
1,77667,A27BPRHBM28XZS,1,3/10/17 8:43,1.489140e+12,X,2638,5000,0.24,0.36,...,0.660000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.00
2,77669,A27BPRHBM28XZS,2,3/10/17 8:44,1.489140e+12,O,1518,5000,0.34,0.64,...,0.380952,0,1,0.375000,0.64,39,21,NaN,0.380952,0.80
3,77670,A27BPRHBM28XZS,3,3/10/17 8:44,1.489140e+12,X,1592,5000,0.24,0.08,...,0.920000,1,0,0.800000,0.92,3,12,NaN,0.714286,0.10
4,77672,A27BPRHBM28XZS,4,3/10/17 8:44,1.489140e+12,X,1397,5000,0.12,0.62,...,0.380000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.00
5,77674,A27BPRHBM28XZS,5,3/10/17 8:44,1.489140e+12,O,4854,5101,0.64,0.72,...,0.600000,0,1,0.666667,0.36,-84,-74,NaN,0.600000,0.06
6,77675,A27BPRHBM28XZS,6,3/10/17 8:44,1.489140e+12,X,1768,5101,0.56,0.48,...,0.250000,0,1,0.200000,0.48,-29,-68,NaN,0.250000,0.20
7,77677,A27BPRHBM28XZS,7,3/10/17 8:44,1.489140e+12,O,1121,5101,0.28,0.76,...,0.363636,0,1,0.350000,0.74,-21,-7,NaN,0.363636,0.40
8,77678,A27BPRHBM28XZS,8,3/10/17 8:44,1.489140e+12,X,2931,5101,0.12,0.40,...,0.580000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.00
9,77680,A27BPRHBM28XZS,9,3/10/17 8:45,1.489140e+12,X,4975,5101,0.56,0.48,...,0.666667,0,1,1.000000,0.48,-29,-68,NaN,0.666667,0.02


In [160]:
tt = trial_table.loc[trial_table['mag_left']>0,]
print(len(tt))
print('no brainers')
no_brainers_left = tt.loc[(tt['prob_out_l_bayes']>tt['prob_out_r_bayes']) & (tt['mag_left']>tt['mag_right']),]
print(len(no_brainers_left))
no_brainers_right = tt.loc[(tt['prob_out_r_bayes']>tt['prob_out_l_bayes']) & (tt['mag_right']>tt['mag_left']),]
print(len(no_brainers_right))
print('mixed trials')
mixed1 =  tt.loc[(tt['prob_out_l_bayes']>tt['prob_out_r_bayes']) & (tt['mag_left']<tt['mag_right']),]
print(len(mixed1))
mixed2 =  tt.loc[(tt['prob_out_l_bayes']<tt['prob_out_r_bayes']) & (tt['mag_left']>tt['mag_right']),]
print(len(mixed2))

150
no brainers
14
23
mixed trials
57
54


In [161]:
print_balance(trial_table)

(mag, prob outcome)

gain
ambig prop urns revealed:0.02
ambig v.s. unambig: (48.92,0.51) v.s. (55.69,0.54)

ambig prop urns revealed:0.06
ambig v.s. unambig: (58.23,0.54) v.s. (46.77,0.62)

ambig prop urns revealed:0.1
ambig v.s. unambig: (46.69,0.56) v.s. (56.31,0.51)

ambig prop urns revealed:0.2
ambig v.s. unambig: (46.92,0.56) v.s. (54.69,0.57)

ambig prop urns revealed:0.4
ambig v.s. unambig: (53.92,0.59) v.s. (56.0,0.62)

ambig prop urns revealed:0.8
ambig v.s. unambig: (31.54,0.67) v.s. (58.08,0.51)

ambig prop urns revealed:1.0
ambig v.s. unambig: (nan,nan) v.s. (nan,nan)

loss
ambig prop urns revealed:0.02
ambig v.s. unambig: (-48.92,0.51) v.s. (-55.69,0.54)

ambig prop urns revealed:0.06
ambig v.s. unambig: (-58.23,0.57) v.s. (-46.77,0.62)

ambig prop urns revealed:0.1
ambig v.s. unambig: (-46.69,0.56) v.s. (-56.31,0.51)

ambig prop urns revealed:0.2
ambig v.s. unambig: (-46.92,0.56) v.s. (-54.69,0.56)

ambig prop urns revealed:0.4
ambig v.s. unambig: (-53.92,0.58) v.s. (-56.

# XO files 
- it's just a big string


In [162]:
with open('colours/colours_toinput_simple_combined.txt', 'r') as myfile:
    data=myfile.read().replace('\n', '')
data[0:100]

'0001101011111101111111011110011101111111101111100110100100010110000000000000000000000001000100010000'

# Tweaking the Balance in Excel 



In [163]:
trial_table.to_csv('parameters/Parameters_block_gain_loss_combined_7_30_2017.csv') # only do this once

- I removed a bunch of columns to keep it simple. 
- The magnitudes are easy to change. 
- Since I'll chance the probabilities too, I'll remake the 00011 files

# Re-Checking after Manual Tweaking

In [5]:
trial_table2 = pd.read_csv('parameters/Parameters_block_gain_loss_combined_7_30_2017_tweaked.csv')
trial_table2 = add_columns(trial_table2)


In [6]:
trial_table2.head()

,trial_number,ml,mr,pr,pl,rr,rl,revealed_o_l,revealed_x_l,revealed_o_r,...,prob_out_l_bayes,prob_out_r_bayes,ambig_l,ambig_r,prob_out_ambig,prob_out_unambig,mag_ambig,mag_unambig,resp_amb_1,prob_out_ambig_bayes
0,0,36,21,0.24,0.84,1.0,1.0,9,41,37,...,0.820000,0.260000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,78,101,0.64,0.76,1.0,1.0,13,37,17,...,0.740000,0.660000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21,68,0.36,0.66,0.8,1.0,18,32,25,...,0.640000,0.380952,0,1,0.375,0.64,68,21,NaN,0.380952
3,3,8,12,0.92,0.76,1.0,0.1,1,4,4,...,0.714286,0.920000,1,0,0.800,0.92,8,12,NaN,0.714286
4,4,7,20,0.38,0.88,1.0,1.0,6,44,31,...,0.880000,0.380000,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Does my pr and pl match reasonable with what's revealed for x's and o's. 

- if prob is 0.66, and info is 0.8 there should be roughly 2/3 x to o's. 
- at the moement, there are

In [7]:
trial_table2.loc[trial_table2.rr==0.8,['rr','pr','revealed_x_r','revealed_o_r']]

,rr,pr,revealed_x_r,revealed_o_r
2,0.8,0.36,15,25
12,0.8,0.36,15,25
25,0.8,0.60,30,20
42,0.8,0.48,18,22
47,0.8,0.48,18,22
80,0.8,0.80,40,10
88,0.8,0.88,25,15
114,0.8,0.60,21,19
115,0.8,0.90,34,6
157,0.8,0.84,31,9


### Number of trials that are split mag pointing one way and prob pointing the other? 

- ambiguous trials 

In [8]:
tt = trial_table2.loc[trial_table2['mag_left']>0,]
print(len(tt))
print('no brainers')
no_brainers_left = tt.loc[(tt['prob_out_l_bayes']>tt['prob_out_r_bayes']) & (tt['mag_left']>tt['mag_right']),]
print(len(no_brainers_left))
no_brainers_right = tt.loc[(tt['prob_out_r_bayes']>tt['prob_out_l_bayes']) & (tt['mag_right']>tt['mag_left']),]
print(len(no_brainers_right))
print('mixed trials (mag/prob point in opposite directions)')
mixed1 =  tt.loc[(tt['prob_out_l_bayes']>tt['prob_out_r_bayes']) & (tt['mag_left']<tt['mag_right']),]
print(len(mixed1))
mixed2 =  tt.loc[(tt['prob_out_l_bayes']<tt['prob_out_r_bayes']) & (tt['mag_left']>tt['mag_right']),]
print(len(mixed2))

150
no brainers
14
24
mixed trials (mag/prob point in opposite directions)
55
53


### Average Balance - per level of ambiguity 

In [11]:
trial_table2 = pd.read_csv('parameters/Parameters_block_gain_loss_combined_7_30_2017_tweaked.csv')
trial_table2 = add_columns(trial_table2)
print_balance(trial_table2)

(mag, prob outcome)

gain
ambig prop urns revealed:0.02
ambig v.s. unambig: (55.38,0.54) v.s. (55.69,0.54)

ambig prop urns revealed:0.06
ambig v.s. unambig: (56.31,0.57) v.s. (56.77,0.57)

ambig prop urns revealed:0.1
ambig v.s. unambig: (53.62,0.56) v.s. (53.77,0.56)

ambig prop urns revealed:0.2
ambig v.s. unambig: (54.0,0.56) v.s. (54.69,0.56)

ambig prop urns revealed:0.4
ambig v.s. unambig: (54.54,0.59) v.s. (54.85,0.59)

ambig prop urns revealed:0.8
ambig v.s. unambig: (52.42,0.58) v.s. (52.83,0.58)

ambig prop urns revealed:1.0
ambig v.s. unambig: (nan,nan) v.s. (nan,nan)

loss
ambig prop urns revealed:0.02
ambig v.s. unambig: (-55.31,0.54) v.s. (-55.69,0.54)

ambig prop urns revealed:0.06
ambig v.s. unambig: (-56.92,0.6) v.s. (-56.31,0.6)

ambig prop urns revealed:0.1
ambig v.s. unambig: (-53.08,0.56) v.s. (-53.08,0.56)

ambig prop urns revealed:0.2
ambig v.s. unambig: (-54.08,0.56) v.s. (-54.69,0.56)

ambig prop urns revealed:0.4
ambig v.s. unambig: (-54.46,0.55) v.s. (-56.0,

# Making loss the same as gain
- so the balancing was kind of unnecessary

In [6]:
trial_table2 = pd.read_csv('parameters/Parameters_block_gain_loss_combined_7_30_2017_tweaked.csv')
trial_table2 = add_columns(trial_table2)
trial_table2.head()

,trial_number,ml,mr,pr,pl,rr,rl,revealed_o_l,revealed_x_l,revealed_o_r,...,prob_out_l_bayes,prob_out_r_bayes,ambig_l,ambig_r,prob_out_ambig,prob_out_unambig,mag_ambig,mag_unambig,resp_amb_1,prob_out_ambig_bayes
0,0,36,21,0.24,0.84,1.0,1.0,9,41,37,...,0.820000,0.260000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,78,101,0.64,0.76,1.0,1.0,13,37,17,...,0.740000,0.660000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21,68,0.36,0.66,0.8,1.0,18,32,25,...,0.640000,0.380952,0,1,0.375,0.64,68,21,NaN,0.380952
3,3,8,12,0.92,0.76,1.0,0.1,1,4,4,...,0.714286,0.920000,1,0,0.800,0.92,8,12,NaN,0.714286
4,4,7,20,0.38,0.88,1.0,1.0,6,44,31,...,0.880000,0.380000,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# replace loss trials with gain trials in some random order
loss_index = trial_table2.loc[trial_table2.ml<0,].index.values
gain_trials = trial_table2.loc[trial_table2.ml>0].sample(frac=1.0).copy()

for loss_trial in range(150):
    # for each row in loss trial, set the row equal to some scrambled row from gain trials.
    row = gain_trials.iloc[loss_trial,].copy()
    row['ml']=row['ml']*-1.0
    row['mr']=row['mr']*-1.0
    row['mag_left']=row['mag_left']*-1.0
    row['mag_right']=row['mag_right']*-1.0
    row['mag_ambig']=row['mag_ambig']*-1.0
    row['mag_unambig']=row['mag_unambig']*-1.0
    trial_table2.loc[loss_index[loss_trial]]= row.as_matrix()
    


In [8]:
trial_table2['trial_number']=np.arange(300)

In [9]:
trial_table2.head(20)

,trial_number,ml,mr,pr,pl,rr,rl,revealed_o_l,revealed_x_l,revealed_o_r,...,prob_out_l_bayes,prob_out_r_bayes,ambig_l,ambig_r,prob_out_ambig,prob_out_unambig,mag_ambig,mag_unambig,resp_amb_1,prob_out_ambig_bayes
0,0,36,21,0.24,0.84,1.00,1.00,9,41,37,...,0.820000,0.260000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,78,101,0.64,0.76,1.00,1.00,13,37,17,...,0.740000,0.660000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21,68,0.36,0.66,0.80,1.00,18,32,25,...,0.640000,0.380952,0,1,0.375000,0.64,68,21,NaN,0.380952
3,3,8,12,0.92,0.76,1.00,0.10,1,4,4,...,0.714286,0.920000,1,0,0.800000,0.92,8,12,NaN,0.714286
4,4,7,20,0.38,0.88,1.00,1.00,6,44,31,...,0.880000,0.380000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,5,-16,-38,0.40,0.74,1.00,1.00,13,37,30,...,0.740000,0.400000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
6,6,-97,-70,0.92,0.64,0.02,1.00,19,31,0,...,0.620000,0.666667,0,1,1.000000,0.62,-70,-97,NaN,0.666667
7,7,-33,-60,0.26,0.42,0.40,1.00,28,22,17,...,0.440000,0.181818,0,1,0.150000,0.44,-60,-33,NaN,0.181818
8,8,-5,-25,0.60,0.96,1.00,1.00,3,47,20,...,0.940000,0.600000,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9,9,-87,-55,0.80,0.40,1.00,0.10,4,1,10,...,0.285714,0.800000,1,0,0.200000,0.80,-87,-55,NaN,0.285714


In [10]:
print_balance(trial_table2)

(mag, prob outcome)

gain
ambig prop urns revealed:0.02
ambig v.s. unambig: (55.38,0.54) v.s. (55.69,0.54)

ambig prop urns revealed:0.06
ambig v.s. unambig: (56.31,0.57) v.s. (56.77,0.57)

ambig prop urns revealed:0.1
ambig v.s. unambig: (53.62,0.56) v.s. (53.77,0.56)

ambig prop urns revealed:0.2
ambig v.s. unambig: (54.0,0.56) v.s. (54.69,0.56)

ambig prop urns revealed:0.4
ambig v.s. unambig: (54.54,0.59) v.s. (54.85,0.59)

ambig prop urns revealed:0.8
ambig v.s. unambig: (52.42,0.58) v.s. (52.83,0.58)

ambig prop urns revealed:1.0
ambig v.s. unambig: (nan,nan) v.s. (nan,nan)

loss
ambig prop urns revealed:0.02
ambig v.s. unambig: (-55.38,0.54) v.s. (-55.69,0.54)

ambig prop urns revealed:0.06
ambig v.s. unambig: (-56.31,0.57) v.s. (-56.77,0.57)

ambig prop urns revealed:0.1
ambig v.s. unambig: (-53.62,0.56) v.s. (-53.77,0.56)

ambig prop urns revealed:0.2
ambig v.s. unambig: (-54.0,0.56) v.s. (-54.69,0.56)

ambig prop urns revealed:0.4
ambig v.s. unambig: (-54.54,0.59) v.s. (-54.8

In [17]:
trial_table2.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12]].to_csv('parameters/Parameters_block_gain_loss_combined_8_21_2017_tweaked_gain_loss_same.csv',index=False)

# Saving out 1's and O's

In [18]:
from IPython.core.debugger import Tracer

In [19]:
def shuffle_o_x_and_justify(input_array=None):
    if input_array is None:
        return(input_array)
    elif len(input_array)<=5:
        return(np.sort(input_array))
    else:
        input_array = input_array.copy()
        np.random.shuffle(input_array)
        new_array = np.array([])
        setsof5=0
        for i in range(len(input_array)):
            i=i+1    
            if np.mod(i,5)==0:
                index = np.arange(setsof5*5,i)
                new_array = np.append(new_array,np.sort(input_array[index]))
                setsof5+=1
        return(new_array)


In [20]:
# testing the function # 
shuffle_o_x_and_justify(np.array([0,1,1]))

array([0, 1, 1])

In [21]:
large_string = ''
for trial in np.sort(trial_table2.trial_number.unique()):
    #print(trial)
    x_right = trial_table2.loc[trial_table2.trial_number==trial,'revealed_x_r'].as_matrix()[0]
    o_right = trial_table2.loc[trial_table2.trial_number==trial,'revealed_o_r'].as_matrix()[0]
    
    o_left = trial_table2.loc[trial_table2.trial_number==trial,'revealed_o_l'].as_matrix()[0]
    x_left = trial_table2.loc[trial_table2.trial_number==trial,'revealed_x_l'].as_matrix()[0]
    
    left_urn_without_eq = np.append(np.zeros(x_left),np.ones(o_left))
    left_urn_without_eq = shuffle_o_x_and_justify(left_urn_without_eq)
    
    right_urn_without_eq = np.append(np.zeros(x_right),np.ones(o_right))
    right_urn_without_eq = shuffle_o_x_and_justify(right_urn_without_eq)
    
    left_urn = np.lib.pad(left_urn_without_eq,(0,50-len(left_urn_without_eq)),'constant',constant_values=2)
    right_urn = np.lib.pad(right_urn_without_eq,(0,50-len(right_urn_without_eq)),'constant',constant_values=2)
    
    # save out to experiment
    #if 2 in right_urn:
    #    right_urn_copy=right_urn_without_eq.copy()
    
    # each trial
    row = np.append(left_urn,right_urn)
    #print(row)
    large_string = large_string+''.join([str(int(r)) for r in row])

/Users/chris/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  # Remove the CWD from sys.path while we load stuff.
/Users/chris/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  del sys.path[0]


In [22]:
# there should be 30,000 items (100 per trial X 300 trials)
print(len(large_string)) 

30000


In [23]:
text_file = open("colours/colours_toinput_simple_combined_8_21_2017.txt", "w")
text_file.write(large_string)
text_file.close()

# Save out for Shock

In [24]:
columns = ['ml','mr','pr','pl','rr','rl'] # this matches shock file
trial_table_gain = trial_table2.loc[trial_table2.ml>0,columns+['trial_number']]



trial_table_gain_mat = trial_table_gain.as_matrix()
trial_table_gain_mat.shape

(150, 7)

In [25]:
trial_table_gain.head()

,ml,mr,pr,pl,rr,rl,trial_number
0,36,21,0.24,0.84,1.0,1.0,0
1,78,101,0.64,0.76,1.0,1.0,1
2,21,68,0.36,0.66,0.8,1.0,2
3,8,12,0.92,0.76,1.0,0.1,3
4,7,20,0.38,0.88,1.0,1.0,4


In [27]:
# write out blocks # 


fname_all = 'parameters/Parameters_order1_8_21_17.txt'

with open(fname_all,'w') as f:
    f.writelines('={{')

    for trial in range(150):
        st = '{'
        for column in range(len(columns)):
            st=st+str(trial_table_gain_mat[trial,column])
            if column<len(columns)-1:
                st = st+','
            else:
                st = st+',0.00}'
                
                
        if trial==49 or trial==99 or trial==149:
            st = st+'}'
        if trial==50 or trial==100:   
            st = '{'+st
            
        if trial != 149:
            st = st+',\n'
        else:
            st=st+'}'
        print('trial')
        print(trial)
        print(st)
        f.writelines(st)

        

trial
0
{36.0,21.0,0.24,0.84,1.0,1.0,0.00},

trial
1
{78.0,101.0,0.64,0.76,1.0,1.0,0.00},

trial
2
{21.0,68.0,0.36,0.66,0.8,1.0,0.00},

trial
3
{8.0,12.0,0.92,0.76,1.0,0.1,0.00},

trial
4
{7.0,20.0,0.38,0.88,1.0,1.0,0.00},

trial
5
{16.0,8.0,0.6,0.96,1.0,1.0,0.00},

trial
6
{98.0,90.0,0.92,0.36,1.0,0.06,0.00},

trial
7
{88.0,80.0,0.68,0.76,0.02,1.0,0.00},

trial
8
{15.0,53.0,0.32,0.96,1.0,1.0,0.00},

trial
9
{22.0,23.0,0.7,0.66,1.0,1.0,0.00},

trial
10
{40.0,7.0,0.72,0.24,1.0,0.2,0.00},

trial
11
{63.0,13.0,0.4,0.48,1.0,1.0,0.00},

trial
12
{88.0,38.0,0.66,0.28,0.06,1.0,0.00},

trial
13
{15.0,5.0,0.94,0.66,1.0,1.0,0.00},

trial
14
{11.0,14.0,0.44,0.44,1.0,0.02,0.00},

trial
15
{68.0,45.0,0.6,0.44,0.8,1.0,0.00},

trial
16
{94.0,100.0,0.52,0.64,1.0,0.4,0.00},

trial
17
{62.0,134.0,0.2,0.26,1.0,1.0,0.00},

trial
18
{27.0,15.0,0.6,0.22,1.0,0.06,0.00},

trial
19
{39.0,34.0,0.88,0.76,1.0,1.0,0.00},

trial
20
{100.0,84.0,0.28,0.36,0.06,1.0,0.00},

trial
21
{68.0,40.0,0.52,0.44,0.2,1.0,0.00},


In [29]:
# write out 0's and 1's # 

fname_all =  'colours/colours_Behaviour_1in5_8_21_17.txt'

with open(fname_all,'w') as f:
    f.writelines('array <int> colours_toinput [3][50][100]; colours_toinput={{')
    for trial,trial_number in enumerate(trial_table_gain.trial_number.sort_values().as_matrix()):

        # get 100 o's and x's for this trial
        string_for_trial = large_string[trial_number*100:(trial_number*100+100)]
    
        # flip the right and the left
        string_for_trial_copy = [s for s in string_for_trial]
        string_for_trial_copy[0:50]=string_for_trial[50::]
        string_for_trial_copy[50::]=string_for_trial[0:50]
        string_for_trial = string_for_trial_copy
        
        st = '{'+str(','.join(string_for_trial))+'}'

        if trial==49 or trial==99 or trial==149:
            st = st+'}'
        if trial==50 or trial==100:   
            st = '{'+st
        if trial != 149:
            st = st+',\n'
        else:
            st=st+'}'
        print('trial')
        print(trial)
        print(st)
        f.writelines(st)
        

trial
0
{1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0},

trial
1
{0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0},

trial
2
{0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,0,1,1,1,0,1,1,1,1,0,0,1,1,1,0,0,1,1,1,2,2,2,2,2,2,2,2,2,2,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1},

trial
3
{0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2},

trial
4
{0,0,0,1,1,0,0,1,1,1,0,1,1,1,1,0,0,0,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,0,1,1,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0

limit_output extension: Maximum message size of 10000 exceeded with 31994 characters